In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Trees_Basic").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 18:49:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import (
    RandomForestClassifier,
    GBTClassifier,
    DecisionTreeClassifier,
)

In [4]:
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

24/03/13 18:52:00 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [8]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [9]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [11]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [12]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,35.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[155,156,180...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(69

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [14]:
acc_eval = MulticlassClassificationEvaluator(metricName="accuracy")

In [15]:
print("DTC_ACCURACY: ")
acc_eval.evaluate(dtc_preds)

DTC_ACCURACY: 


0.9428571428571428

In [16]:
rfc_model.featureImportances

SparseVector(692, {132: 0.0004, 154: 0.0002, 159: 0.0008, 176: 0.0006, 185: 0.0006, 188: 0.0012, 204: 0.0023, 207: 0.0019, 208: 0.0021, 209: 0.0005, 216: 0.0218, 238: 0.0009, 242: 0.0009, 243: 0.0008, 244: 0.0328, 245: 0.0067, 260: 0.0026, 263: 0.0067, 271: 0.0138, 272: 0.0278, 273: 0.0096, 286: 0.0015, 290: 0.0073, 291: 0.0006, 293: 0.0003, 297: 0.0006, 298: 0.0006, 300: 0.0501, 301: 0.0069, 317: 0.008, 318: 0.0093, 319: 0.0006, 321: 0.001, 322: 0.0007, 323: 0.0019, 329: 0.011, 341: 0.0018, 343: 0.0008, 345: 0.0113, 347: 0.0012, 350: 0.0181, 351: 0.0189, 355: 0.0042, 356: 0.0088, 357: 0.0233, 358: 0.0079, 359: 0.0006, 370: 0.0006, 373: 0.006, 374: 0.0062, 375: 0.0017, 377: 0.002, 378: 0.0705, 379: 0.0203, 382: 0.0051, 385: 0.0074, 401: 0.0057, 402: 0.0013, 403: 0.0015, 405: 0.0294, 406: 0.0534, 407: 0.0486, 411: 0.0011, 412: 0.0002, 413: 0.0052, 427: 0.0027, 429: 0.0022, 433: 0.0138, 434: 0.0181, 435: 0.0017, 438: 0.0052, 439: 0.0011, 440: 0.0034, 454: 0.0105, 456: 0.0127, 457: 0.0056